# 羽毛球3D轨迹可视化（示例与说明）
本Notebook演示如何加载数据、绘制标准3D羽毛球场地、可视化多条轨迹，并播放动画。

In [8]:
!pip -q install plotly pandas numpy 'nbformat>=4.2.0'


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


## 1. 导入与依赖

In [9]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import sys
# sys.path.append(str(Path('.').resolve()))
# sys.path.append('./')
from badminton_vis import (
    CourtSpec, load_from_json_file, load_from_json_obj, load_from_dataframe,
    sort_and_clean, smooth, interpolate_time, make_figure, make_animation,
    save_html, make_parabola_example
)

## 2. 数据格式与示例
示例JSON位于 `/mnt/data/badminton_3d_vis/sample_trajectory.json`，符合规格中的字段定义。

In [10]:
sample_path = './sample_trajectory.json'
with open(sample_path, 'r', encoding='utf-8') as f:
    sample_obj = json.load(f)
sample_obj


{'msgtype': 'real_track',
 'pid': '50651a30',
 'positions': [{'frame_id': 23070,
   'left': {'x': 946.0, 'y': 593.0},
   'pos': {'x': 839.7114868164062,
    'y': -5.297586441040039,
    'z': 137.5612335205078},
   'right': {'x': 943.0, 'y': 598.0},
   'ts': 1756378659555.455,
   'vx': 0.0,
   'vy': 0.0,
   'vz': 0.0},
  {'frame_id': 23071,
   'left': {'x': 948.0, 'y': 581.0},
   'pos': {'x': 833.2072143554688,
    'y': -5.916860103607178,
    'z': 146.10671997070312},
   'right': {'x': 943.0, 'y': 586.0},
   'ts': 1756378659561.714,
   'vx': 0.0,
   'vy': 0.0,
   'vz': 0.0},
  {'frame_id': 23072,
   'left': {'x': 951.0, 'y': 571.0},
   'pos': {'x': 823.550537109375,
    'y': -6.9140753746032715,
    'z': 152.14077758789062},
   'right': {'x': 943.0, 'y': 578.0},
   'ts': 1756378659567.949,
   'vx': 0.0,
   'vy': 0.0,
   'vz': 0.0},
  {'frame_id': 23074,
   'left': {'x': 956.0, 'y': 551.0},
   'pos': {'x': 817.30419921875,
    'y': -9.639930725097656,
    'z': 167.06744384765625},
   'r

## 3. 加载与解析
支持从JSON对象/文件或DataFrame加载，自动生成 `pid, frame_id, ts, x, y, z, vx, vy, vz` 字段。

In [11]:
trajs = load_from_json_file(sample_path)
df = pd.concat([t.to_dataframe() for t in trajs], ignore_index=True)
df = sort_and_clean(df)
df.head()


,pid,frame_id,ts,x,y,z,vx,vy,vz
0,50651a30,23070,1.756379e+12,839.711487,-5.297586,137.561234,0.0,0.0,0.0
1,50651a30,23071,1.756379e+12,833.207214,-5.916860,146.106720,0.0,0.0,0.0
2,50651a30,23072,1.756379e+12,823.550537,-6.914075,152.140778,0.0,0.0,0.0
3,50651a30,23074,1.756379e+12,817.304199,-9.639931,167.067444,0.0,0.0,0.0
4,50651a30,23075,1.756379e+12,817.470642,-9.552011,173.651672,0.0,0.0,0.0


## 4. 绘制静态3D场地与轨迹

In [12]:
court = CourtSpec()  # 可传入自定义尺寸
fig = make_figure(df, court=court, title='Badminton 3D Trajectories - Static')
fig.show()


## 5. 轨迹动画（根据时间戳ts）

In [13]:
anim_fig = make_animation(df, court=CourtSpec(), title='Badminton 3D Trajectory Animation', time_col='ts')
anim_fig.show()


## 6. 多轨迹演示与平滑/插值
这里创建另一条仿真轨迹并与示例轨迹一起显示，同时展示平滑与按固定时间步长插值。

In [14]:
# 生成另一条抛物线轨迹
demo_trajs = make_parabola_example(pid='50651a30', apex=(0.0, -1.0, 6.0), span_x=7.0)
df2 = pd.concat([t.to_dataframe() for t in demo_trajs], ignore_index=True)
df_multi = pd.concat([df, df2], ignore_index=True)
df_multi = sort_and_clean(df_multi)

# 可选：平滑
df_smooth = smooth(df_multi, window=5)

# 可选：按固定时间步插值（需要ts列）
df_interp = interpolate_time(df_smooth, freq_ms=20)

fig_multi = make_figure(df_interp, court=CourtSpec(), title='Multiple Trajectories (Smoothed & Interpolated)')
fig_multi.show()


/tmp/ipykernel_2321301/1364876385.py:4: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



MemoryError: Unable to allocate 654. GiB for an array with shape (87818933065,) and data type float64

## 7. 导出（HTML）

In [ ]:
export_path = '/mnt/data/badminton_3d_vis/trajectory_visualization.html'
save_html(fig_multi, export_path)
export_path


## 8. 常见问题（FAQ）
- **坐标单位**：默认以米为单位。
- **坐标系**：原点在场地中心地面，x为场地长度方向，y为宽度方向，z向上。
- **自定义场地**：通过 `CourtSpec(length=..., width_doubles=..., net_height_center=...)` 调整。
- **数据来源**：可使用 `load_from_json_file/json_obj/dataframe` 加载；DataFrame方式可映射自定义列名。
- **动画时间**：优先使用 `ts`（毫秒）；如缺失，将按采样序号合成时间。
- **导出**：使用 `save_html(fig, path)` 导出可交互HTML，便于分享。